# Task 4.7: Deriving New Variables

# CAUTION!  I've moved to an EC2 instance with 32 GiB of RAM - and the option to add more RAM pretty cheaply - so I can afford to work with whatever size DataFrame this course is likely to use, without needing to carve out subsets or make any optimizations.  Please let me know if this poses a problem in evaluating my submissions.

## Reusing notebook from Exercise 4.7, as suggested by the Task instructions.
## In turn, this incorporates the prior Notebook for Task 4.6, Steps #3-8 to pre-load data.

# See "Beginning TASK 4.7" - far down! - to review this task submission.

In [1]:
# Old stuff here, first, with the addition of Execution Time information.

## Step 3:  In a new notebook, import the orders_products_combined dataframe from the pickle file you just saved.

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [3]:
# Starting with Exercise 4.7, import additional libraries I've discovered thanks to Lubov, especially the time library.
import time
import string
import random

In [4]:
# Set a PATH variable.

In [5]:
# Linux PATH - Sadly, my Linux system with 8GB of total RAM could not import the new df_ords_prior DataFrame.
# BUT, now I have moved again, to an AWS EC2 instance that should be able to handle this.

In [6]:
path = (r'~/careerfoundry/tasks/data-immersion/Achievement4/Instacart-Basket-Analysis')

In [7]:
# windoze PATH - The windoze laptop I bought for Excel and Tableau also has 16GB of RAM, so I migrated there.

In [8]:
## path = (r'C:\Users\emccc\OneDrive\Documents\CareerFoundry\Achievement-4\Instacart-Basket-Analysis')

In [9]:
# Read the orders_products_combined.pkl file into a new DataFrame, df_orders_products_combined .

In [10]:
df_orders_products_combined = pd.read_pickle(os.path.join(path, '02-Data', 'Prepared-Data', 'orders_products_combined.pkl'))

## Step #4:  Check the shape of the imported dataframe (it should be the same as the one you exported—always check!).

In [11]:
# Basic checking for comparison to the previous incarnation as, "df_merged_large"

In [12]:
df_orders_products_combined.head()

,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,0.0,196,1,0,both
1,2539329,1,1,2,8,0.0,14084,2,0,both
2,2539329,1,1,2,8,0.0,12427,3,0,both
3,2539329,1,1,2,8,0.0,26088,4,0,both
4,2539329,1,1,2,8,0.0,26405,5,0,both


In [13]:
df_orders_products_combined.shape

(32434489, 10)

In [14]:
df_orders_products_combined['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

## Step #5:  Determine a suitable way to combine the orders_products_combined dataframe with your products data set. Make sure you’re using your wrangled, cleaned, and deduped products data set stored in your “Prepared Data” folder from the previous Exercise’s task.

In [15]:
# Read the DataFrame 'df_prods_checked' from the file products_checked.csv .

In [16]:
df_prods_checked = pd.read_csv(os.path.join(path, '02-Data','Prepared-Data', 'products_checked.csv'))

In [17]:
# Sanity checks for df_prods_checked .

In [18]:
# NOTE:  In the previous Exercise/Task, the df_prods_checked DataFrame was written to a CSV file with an additional index
# column by default (Exercise 4.4 did not mention the 'index = False' option to 'df.to_csv').  I'll see whether this
# poses a true problem, as it might not.

In [19]:
df_prods_checked.shape

(49672, 6)

In [20]:
df_prods_checked

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3
...,...,...,...,...,...,...
49667,49688,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,5.3
49668,49689,49685,En Croute Roast Hazelnut Cranberry,42,1,3.1
49669,49690,49686,Artisan Baguette,112,3,7.8
49670,49691,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,4.7


In [21]:
df_prods_checked.describe()

,Unnamed: 0,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000,49672.000000
mean,24852.005053,24850.349775,67.762442,11.728942,9.993282
std,14342.265579,14340.705287,38.315784,5.850779,453.615536
min,0.000000,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,12432.750000,35.000000,7.000000,4.100000
50%,24851.500000,24850.500000,69.000000,13.000000,7.100000
75%,37272.250000,37268.250000,100.000000,17.000000,11.100000
max,49692.000000,49688.000000,134.000000,21.000000,99999.000000


In [22]:
# Combine df_prods_checked with df_orders_products_combined to create the 'orders_products_merged' DataFrame.

In [23]:
# NOTE:  If I want to use an Indicator, I need to give the indicator column a new name, because '_merge' is already
# used in the df_orders_products_combined DataFrame.  In a spark of creativity, I call it, '_merge2'.

In [24]:
df_orders_products_merged = df_prods_checked.merge(df_orders_products_combined, on = 'product_id', indicator = '_merge2')

In [25]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both


In [26]:
df_orders_products_merged.shape

(32404859, 16)

## Step 6:  Confirm the results of the merge using the merge flag.

In [27]:
# Tally up the results from the '_merge2' column.  It's from an inner join, so all should be 'both',
# but this is how I interpret the instruction based on the Exercise.

In [28]:
df_orders_products_merged['_merge2'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge2, dtype: int64

## Step 7:  Export the newly created dataframe as orders_products_merged in a suitable format (taking into consideration the size).

In [29]:
# Store the new DataFrame in the Pickle format.

In [30]:
df_orders_products_merged.to_pickle(os.path.join(path, '02-Data','Prepared-Data', 'orders_products_merged.pkl'))

# Beginning EXERCISE 4.7

In [31]:
# Confirming our DF has been loaded (I know, it's in the code above.)
# Exercise 4.7 seems to be calling this 'ords_prods_merged'.

In [32]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both


In [33]:
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [34]:
# Holy crap, adding a column to this whole DataFrame in place takes over seven minutes and north of 12GB of memory on my EC2 instance.
# It's no wonder the Exercise suggests working with just the first million rows.

In [35]:
df_orders_products_merged['price_range'] = df_orders_products_merged.apply(price_label, axis=1)

In [36]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2,price_range
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product


In [37]:
df_orders_products_merged.shape

(32404859, 17)

In [38]:
df_orders_products_merged['price_range'].value_counts(dropna = False)

Mid-range product    21860860
Low-range product    10126321
High range             417678
Name: price_range, dtype: int64

In [39]:
# A maximum price of precisely 99999.0 sounds like there could be placeholders in play.

In [40]:
df_orders_products_merged['prices'].max()

99999.0

In [41]:
# Even holier crap:  the built-in loc() function is nearly 150 times faster than our User-Defined Function (UDF).

In [42]:
df_orders_products_merged.loc[df_orders_products_merged['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [43]:
df_orders_products_merged.loc[(df_orders_products_merged['prices'] <= 15) & (df_orders_products_merged['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [44]:
 df_orders_products_merged.loc[df_orders_products_merged['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [45]:
df_orders_products_merged['price_range_loc'].value_counts(dropna = False)

Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: price_range_loc, dtype: int64

In [46]:
df_orders_products_merged.shape

(32404859, 18)

In [47]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2,price_range,price_range_loc
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Mid-range product
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Mid-range product
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Mid-range product
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product,Mid-range product
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Mid-range product


In [48]:
# The following is a very interesting way of building a new column, but I must say I think it makes
# more sense to combine the following steps, i.e. append the new column to each row as we process it.

In [49]:
# Step #1:  Build the new column, 'result', as a list.  Note that my DF has an 'order_dow' column, rather than
# an 'orders_day_of_week' column.  Have I mixed up DFs somewhere. or is the Exercise author confused?
# ANSWER:  YES, I MESSED UP IN PREVIOUS SCRIPTS AND RELOADED THE ORIGINAL, NON-WRANGLED orders.csv.  I am fixing this all now.

In [50]:
result = []

for value in df_orders_products_merged["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [51]:
# Step #2:  Tack the set of results on to the DF as a new column, 'busiest_day'.

In [52]:
df_orders_products_merged['busiest_day'] = result

In [53]:
df_orders_products_merged['busiest_day'].value_counts(dropna = False)

Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: busiest_day, dtype: int64

In [54]:
df_orders_products_merged['orders_day_of_week'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: orders_day_of_week, dtype: int64

In [55]:
df_orders_products_merged.shape

(32404859, 19)

In [56]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2,price_range,price_range_loc,busiest_day
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Mid-range product,Regularly busy
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Mid-range product,Regularly busy
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Mid-range product,Busiest day
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product,Mid-range product,Regularly busy
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Mid-range product,Least busy


# Beginning TASK 4.7

## Step #1:  If you haven’t done so already, complete the instructions in the Exercise for creating the “price_label” and “busiest_day” columns.

### Done during the Exercise, above.
### I think the “price_label” column is really supposed to be the "price_range" column.  “price_label” is the name of the User Defined Function that was used to generate the "price_range" column.

## Step #2:  Suppose your clients have changed their minds about the labels you created in your “busiest_day” column. Now, they want “Busiest day” to become “Busiest days” (plural). This label should correspond with the two busiest days of the week as opposed to the single busiest day. At the same time, they’d also like to know the two slowest days. Create a new column for this using a suitable method.

### Rather than create Yet Another column for this big DataFrame as instructed, I'm going to rename the column and fill it with the new set of values in place, to save on memory.  This not only seems more sensible and practical, it appears to be more in accord with the task rubric, which says, '“busiest_day” updated to “busiest_days”'.

In [57]:
# Rename the existing column.

In [58]:
df_orders_products_merged.rename(columns={"busiest_day": "busiest_days"}, inplace=True)

In [59]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2,price_range,price_range_loc,busiest_days
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Mid-range product,Regularly busy
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Mid-range product,Regularly busy
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Mid-range product,Busiest day
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product,Mid-range product,Regularly busy
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Mid-range product,Least busy


In [60]:
# Re-use our previous method - even if a two-step method seems inefiicient to me - to fill in the new values.

In [61]:
# Step A:  Build the new column, 'result', as a list.  Note that my DF has an 'order_dow' column, rather than
# an 'orders_day_of_week' column.  Have I mixed up DFs somewhere. or is the Exercise author confused?
# ANSWER:  YES, I MESSED UP IN PREVIOUS SCRIPTS AND RELOADED THE ORIGINAL, NON-WRANGLED orders.csv.  I am fixing this all now.

In [62]:
result = []

for value in df_orders_products_merged["orders_day_of_week"]:
  if (value == 0 or value == 1):
    result.append("Busiest days")
  elif (value == 4 or value == 3):
    result.append("Least busy days")
  else:
    result.append("Regularly busy")

In [63]:
# Step B:  Tack the set of results on to the DataFrame column, 'busiest_days'.

In [64]:
df_orders_products_merged['busiest_days'] = result

## Step #3:  Check the values of this new column for accuracy. Note any observations in markdown format.

### The value_counts results have shifted, as expected, but the totals still add up to the same sum as before, and all other results appear as expected.

In [65]:
df_orders_products_merged['busiest_days'].value_counts(dropna = False)

Regularly busy     12916111
Busiest days       11864412
Least busy days     7624336
Name: busiest_days, dtype: int64

In [66]:
df_orders_products_merged['orders_day_of_week'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: orders_day_of_week, dtype: int64

In [67]:
df_orders_products_merged.shape

(32404859, 19)

In [68]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2,price_range,price_range_loc,busiest_days
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Mid-range product,Regularly busy
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Mid-range product,Regularly busy
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Mid-range product,Busiest days
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product,Mid-range product,Least busy days
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Mid-range product,Least busy days


## Step #4:  When too many users make Instacart orders at the same time, the app freezes. The senior technical officer at Instacart wants you to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day.”

### To be realistic, we should look at the busiest hours of the whole week (or even month), rather than pretending that every day of the week has the same pattern, but this is just an exercise in Python, after all.

### NOTE:  I choose to interpret these instructions similarly to how the busiest days were determined in Step #2, i.e. I'll choose the TWO busiest hours and the TWO least-busy hours.

In [69]:
# Using the same method as Step #2:  recycling code with replacments.
# First, though, we need to find the busiest and least-busy hours:

In [70]:
df_orders_products_merged['order_hour_of_day'].value_counts(dropna = False)

10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: order_hour_of_day, dtype: int64

In [71]:
# Step A:  Build the new column, 'result', as a list.

In [72]:
result = []

for value in df_orders_products_merged["order_hour_of_day"]:
  if (value == 10 or value == 11):
    result.append("Busiest hours")
  elif (value == 4 or value == 3): # Huh.  The least-busy days are also the least-busy hours.
    result.append("Least busy hours")
  else:
    result.append("Regularly busy")

In [73]:
# Step B:  Tack the set of results on to the DataFrame column, 'busiest_hours'.

In [74]:
df_orders_products_merged['busiest_hours'] = result

## Step #5:  Print the frequency for this new column.

### The numbers total the same expected sum (32,404,859).

In [75]:
df_orders_products_merged['busiest_hours'].value_counts(dropna = False)

Regularly busy      26802436
Busiest hours        5497900
Least busy hours      104523
Name: busiest_hours, dtype: int64

In [76]:
# A couple of extra peeks, just for fun.  I expect a 20th column here.

In [77]:
df_orders_products_merged.shape

(32404859, 20)

In [78]:
df_orders_products_merged.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,_merge2,price_range,price_range_loc,busiest_days,busiest_hours
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Mid-range product,Regularly busy,Busiest hours
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Mid-range product,Busiest days,Regularly busy
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,0.0,10,0,both,both,Mid-range product,Mid-range product,Least busy days,Regularly busy
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Mid-range product,Least busy days,Regularly busy


## Step #6:  Ensure your notebook is clean and structured and that your code is well commented.

### :fingers_crosssed:

## Step #7:  Export your dataframe as a pickle file (since you added new columns) and store it correctly in your “Prepared Data” folder.

In [79]:
df_orders_products_merged.to_pickle(os.path.join(path, '02-Data','Prepared-Data', 'orders_products_merged.pkl'))